In [288]:
import numpy as np # библиотека для работы с чиселками
import pandas as pd # data processing, работа с CSV файлами
import matplotlib.pyplot as plt # для графики
import seaborn as sns # аналогично

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

Откроем датасет и посмотрим первые 5 его строчек

In [289]:
dataset = pd.read_csv('laptop_price.csv', encoding='ISO-8859-1')
dataset.head(5)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


In [290]:
dataset.describe()

,laptop_ID,Inches,Price_euros
count,1303.000000,1303.000000,1303.000000
mean,660.155794,15.017191,1123.686992
std,381.172104,1.426304,699.009043
min,1.000000,10.100000,174.000000
25%,331.500000,14.000000,599.000000
50%,659.000000,15.600000,977.000000
75%,990.500000,15.600000,1487.880000
max,1320.000000,18.400000,6099.000000


Т.к. необходимо решить задачу классификации, то разделим цены на 3 категории: низкие, средние и высокие

Для задачи регрессии будем предсказывать цену ноутбука по остальным характеристикам

In [291]:
dataset['Price_category'] = pd.qcut(dataset['Price_euros'], q=3, labels=['Low', 'Medium', 'High'])


Подготовим данные

In [292]:
# Удаление ненужных столбцов
dataset = dataset.drop(columns=['laptop_ID'])
new_dataset = dataset.copy()

# Кодирование категориальных признаков
categorical_columns = ['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight']
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    new_dataset[column] = le.fit_transform(new_dataset[column])
    label_encoders[column] = le  # Сохраняем энкодеры, если понадобится обратное преобразование


# Разделение на признаки и целевую переменную
X = new_dataset.drop(columns=['Price_euros', 'Price_category'])
y_classification = new_dataset['Price_category']
y_regression = new_dataset['Price_euros']

# Масштабирование числовых признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


Разделим датасет на обучающую и тестовую выборку

In [293]:
X_train, X_test, y_train_classification, y_test_classification = train_test_split(X_scaled, y_classification, test_size=0.2, random_state=42)

In [294]:
X_train.shape

(1042, 11)

In [295]:
y_train_classification


10         Low
147        Low
1287    Medium
767     Medium
816       High
         ...  
1095    Medium
1130       Low
1294       Low
860     Medium
1126       Low
Name: Price_category, Length: 1042, dtype: category
Categories (3, object): ['Low' < 'Medium' < 'High']

Обучим модель Случайного леса

In [296]:
model_classification = RandomForestClassifier(
    n_estimators=100,            # Количество деревьев в лесу
    max_depth=15,                # Максимальная глубина деревьев
    min_samples_split=2,         # Минимальное число объектов для разделения узла
    min_samples_leaf=1,          # Минимальное число объектов в листе
    random_state=42,             # Фиксируем генерацию случайных чисел
    n_jobs=-1                    # Использовать все ядра процессора
)

model_classification.fit(X_train, y_train_classification)

RandomForestClassifier(max_depth=15, n_jobs=-1, random_state=42)

Оценим работу классификатора

In [297]:
train_predict_classification = model_classification.predict(X_train)
test_predict_classification = model_classification.predict(X_test)

In [298]:
# Расчет метрик
train_f1_classification = f1_score(y_train_classification, train_predict_classification, average='micro')
train_accuracy_classification = accuracy_score(y_train_classification, train_predict_classification)
train_precision_classification = precision_score(y_train_classification, train_predict_classification, average='micro')
train_recall_classification = recall_score(y_train_classification, train_predict_classification, average='micro')

test_f1_classification = f1_score(y_test_classification, test_predict_classification, average='micro')
test_accuracy_classification = accuracy_score(y_test_classification, test_predict_classification)
test_precision_classification = precision_score(y_test_classification, test_predict_classification, average='micro')
test_recall_classification = recall_score(y_test_classification, test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', train_f1_classification, train_accuracy_classification, train_precision_classification, train_recall_classification)
print('Test', test_f1_classification, test_accuracy_classification, test_precision_classification, test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.9932821497120922 0.9932821497120922 0.9932821497120922 0.9932821497120922
Test 0.8582375478927203 0.8582375478927203 0.8582375478927203 0.8582375478927203


### Теперь посмотрим работу Random forest алгоритма для задачи регрессии

Разделим датасет на обучающую и тестовую выборку

In [299]:
X_train, X_test, y_train_regression, y_test_regression = train_test_split(X_scaled, y_regression, test_size=0.2, random_state=42)

Обучим модель

In [300]:
model_regression = RandomForestRegressor(
    n_estimators=100,            # Количество деревьев в лесу
    max_depth=15,                # Максимальная глубина деревьев
    min_samples_split=2,         # Минимальное число объектов для разделения узла
    min_samples_leaf=1,          # Минимальное число объектов в листе
    random_state=42,             # Фиксируем генерацию случайных чисел
    n_jobs=-1                    # Использовать все ядра процессора
)

model_regression.fit(X_train, y_train_regression)

RandomForestRegressor(max_depth=15, n_jobs=-1, random_state=42)

Оценим работу

In [301]:
train_predict_regression = model_regression.predict(X_train)
test_predict_regression = model_regression.predict(X_test)

In [302]:
# Расчет метрик
train_mse_regression = mean_squared_error(y_train_regression, train_predict_regression)
train_mae_regression = mean_absolute_error(y_train_regression, train_predict_regression)

test_mse_regression = mean_squared_error(y_test_regression, test_predict_regression)
test_mae_regression = mean_absolute_error(y_test_regression, test_predict_regression)



print('Метрика:          MSE            MAE')
print('Train', train_mse_regression, train_mae_regression)
print('Test', test_mse_regression, test_mae_regression)

Метрика:          MSE            MAE
Train 13473.774323496078 70.22590624118544
Test 80069.28913126278 166.79491251105034


### Вывод:
В задаче классификации случайный лес показал высокую производительность на тренировочной выборке, что подтверждается почти идеальными значениями метрик F1, Accuracy, Precision и Recall (0.993). Однако на тестовой выборке метрики значительно ниже (0.858), что указывает на возможное переобучение модели. Модель хорошо справляется с задачей, но может быть улучшена за счёт оптимизации гиперпараметров, таких как количество деревьев, максимальная глубина, или минимальное количество объектов в листе.

В задаче регрессии на тренировочной выборке модель достигла удовлетворительных значений MSE (13,473.77) и MAE (70.23), что указывает на точные предсказания. Однако на тестовой выборке MSE выросло до 80,069.29, а MAE до 166.79, что также может свидетельствовать о некотором переобучении.



# Улучшение бейзлайна

Для улучшения бейзлайна модели в задачах классификации и регрессии предлагаю следующие решения:

Удалить столбцы с высокой кореляцией, закодировать категориальные признаки более информативно (используя One-Hot Encoding), а также использовать метод GridSearchCV для поиска лучших параметров.

Для начала посмотрим матрицу кореляций для данного датасета

In [303]:
X.corr()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
Company,1.000000,0.067234,-0.007718,-0.085784,0.094772,0.044593,-0.008399,0.048086,0.026537,0.134290,-0.159906
Product,0.067234,1.000000,0.065594,-0.218974,0.176393,0.143855,0.018305,0.088667,0.094795,0.120562,-0.204379
TypeName,-0.007718,0.065594,1.000000,-0.077428,-0.175618,-0.128174,0.213988,0.019091,-0.204774,0.085223,-0.211832
Inches,-0.085784,-0.218974,-0.077428,1.000000,-0.247841,0.153041,-0.149706,-0.193382,0.215710,0.034528,0.866304
ScreenResolution,0.094772,0.176393,-0.175618,-0.247841,1.000000,0.232751,0.019858,0.059042,0.160816,0.070360,-0.218135
Cpu,0.044593,0.143855,-0.128174,0.153041,0.232751,1.000000,-0.077176,-0.013264,0.490761,0.125374,0.139861
Ram,-0.008399,0.018305,0.213988,-0.149706,0.019858,-0.077176,1.000000,-0.248580,-0.138212,0.008627,-0.174577
Memory,0.048086,0.088667,0.019091,-0.193382,0.059042,-0.013264,-0.248580,1.000000,-0.020784,0.035574,-0.218299
Gpu,0.026537,0.094795,-0.204774,0.215710,0.160816,0.490761,-0.138212,-0.020784,1.000000,0.103853,0.274961
OpSys,0.134290,0.120562,0.085223,0.034528,0.070360,0.125374,0.008627,0.035574,0.103853,1.000000,-0.031506


Можно заметить, что у Weight и Inches коэффициент кореляции 0.86, что достаточно много, значит уберем один из этих столбцов (пусть это будет Weight)

In [304]:
dataset = dataset.drop(columns=['Weight'])

In [305]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [306]:
# Упрощенный пайплайн
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_features)
    ])

X_train_preprocessed = preprocessor.fit_transform(new_X_train)
X_test_preprocessed = preprocessor.transform(new_X_test)

classification_pipeline = RandomForestClassifier()
regression_pipeline = RandomForestRegressor()

param_grid_classification = {
    'n_estimators': [10, 50],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'criterion': ['gini', 'entropy']
}

param_grid_regression = {
    'n_estimators': [10, 50],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'criterion': ['squared_error', 'absolute_error']
}

grid_search_classification = RandomizedSearchCV(classification_pipeline, param_grid_classification, n_iter=10, cv=3, random_state=42)
grid_search_classification.fit(X_train_preprocessed, y_train_classification)

grid_search_regression = RandomizedSearchCV(regression_pipeline, param_grid_regression, n_iter=10, cv=3, random_state=42)
grid_search_regression.fit(X_train_preprocessed, y_train_regression)

print("Best parameters for classification:", grid_search_classification.best_params_)
print("Best parameters for regression:", grid_search_regression.best_params_)


Best parameters for classification: {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20, 'criterion': 'entropy'}
Best parameters for regression: {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20, 'criterion': 'absolute_error'}


In [307]:
preprocessor

ColumnTransformer(transformers=[('num', StandardScaler(), ['Inches']),
                                ('cat',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['Company', 'Product', 'TypeName',
                                  'ScreenResolution', 'Cpu', 'Ram', 'Memory',
                                  'Gpu', 'OpSys'])])

In [308]:
preprocessor.fit(new_X_train)

X_train_preprocessed = preprocessor.transform(new_X_train)
X_test_preprocessed = preprocessor.transform(new_X_test)

Обучим модели и оценим их работу

In [309]:
new_model_classification = RandomForestClassifier(n_estimators=50, min_samples_split=2, min_samples_leaf=1, max_depth=20, criterion='entropy')

new_model_classification.fit(X_train_preprocessed, y_train_classification)

RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=50)

In [310]:
new_model_regression = RandomForestRegressor(n_estimators=50, min_samples_split=2, min_samples_leaf=1, max_depth=20, criterion='absolute_error')

new_model_regression.fit(X_train_preprocessed, y_train_regression)

RandomForestRegressor(criterion='absolute_error', max_depth=20, n_estimators=50)

In [311]:
new_train_predict_classification = new_model_classification.predict(X_train_preprocessed)
new_test_predict_classification = new_model_classification.predict(X_test_preprocessed)

In [312]:
# Расчет метрик
new_train_f1_classification = f1_score(y_train_classification, new_train_predict_classification, average='micro')
new_train_accuracy_classification = accuracy_score(y_train_classification, new_train_predict_classification)
new_train_precision_classification = precision_score(y_train_classification, new_train_predict_classification, average='micro')
new_train_recall_classification = recall_score(y_train_classification, new_train_predict_classification, average='micro')

new_test_f1_classification = f1_score(y_test_classification, new_test_predict_classification, average='micro')
new_test_accuracy_classification = accuracy_score(y_test_classification, new_test_predict_classification)
new_test_precision_classification = precision_score(y_test_classification, new_test_predict_classification, average='micro')
new_test_recall_classification = recall_score(y_test_classification, new_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', new_train_f1_classification, new_train_accuracy_classification, new_train_precision_classification, new_train_recall_classification)
print('Test', new_test_f1_classification, new_test_accuracy_classification, new_test_precision_classification, new_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.9961612284069098 0.9961612284069098 0.9961612284069098 0.9961612284069098
Test 0.7777777777777778 0.7777777777777778 0.7777777777777778 0.7777777777777778


In [313]:
new_train_predict_regression = new_model_regression.predict(X_train_preprocessed)
new_test_predict_regression = new_model_regression.predict(X_test_preprocessed)

In [314]:
# Расчет метрик
new_train_mse_regression = mean_squared_error(y_train_regression, new_train_predict_regression)
new_train_mae_regression = mean_absolute_error(y_train_regression, new_train_predict_regression)

new_test_mse_regression = mean_squared_error(y_test_regression, new_test_predict_regression)
new_test_mae_regression = mean_absolute_error(y_test_regression, new_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', new_train_mse_regression, new_train_mae_regression)
print('Test', new_test_mse_regression, new_test_mae_regression)

Метрика:          MSE            MAE
Train 13191.992693243144 69.93938186180421
Test 178721.92758458582 227.34208544061303


### Вывод:
До улучшения на обучающей выборке алгоритм показывал высокие значения всех метрик, что свидетельствует о практически идеальной классификации на тренировочных данных: 0.993. Метрики регрессии также показывают хорошую точность: MSE = 13473.77, MAE = 70.23. Метрики классификации на тестовой выборке немного хуже, чем на обучении (F1 = 0.858), но все еще высокие. Метрики регрессии значительно хуже: MSE = 80069.29, MAE = 166.79, что может указывать на переобучение модели.

После улучшения метрики классификации на обучающей выборке улучшились незначительно: F1 = 0.997, однако это практически неотличимо от первоначального результата. Метрики регрессии немного снизились: MSE = 12778.66, MAE = 71.10, что указывает на более устойчивую модель. Качество на тестовой выборке снизилось: для классификации F1 = 0.808, что говорит о чуть меньшей обобщающей способности, а для регрессии метрики заметно ухудшились: MSE = 171348.58, MAE = 227.63, что, возможно, связано с усложнением модели или недостаточной оптимизацией параметров.

Общий вывод: до улучшения модель классификации хорошо обобщалась на тестовой выборке, но модель регрессии страдала от переобучения. Улучшение модели привело к лучшим результатам на обучении, но слегка ухудшило качество на тестовой выборке, особенно в задаче регрессии.

### Имплементация алгоритма

In [315]:
from collections import Counter
from sklearn.base import BaseEstimator

In [316]:
class GradientBoostingClassifier(BaseEstimator):
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, random_state=None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.trees = []
        self.classes_ = np.unique(y)
        self.init_prediction = np.log(np.mean(y) / (1 - np.mean(y)))
        predictions = np.full(len(y), self.init_prediction)

        for _ in range(self.n_estimators):
            # Compute the pseudo residuals
            residuals = y - 1 / (1 + np.exp(-predictions))

            # Fit a weak learner (decision tree) to the residuals
            from sklearn.tree import DecisionTreeRegressor
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(X, residuals)

            # Update the predictions
            update = self.learning_rate * tree.predict(X)
            predictions += update

            # Store the tree
            self.trees.append(tree)

    def predict_proba(self, X):
        # Compute the final predictions
        predictions = np.full(X.shape[0], self.init_prediction)
        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(X)
        # Convert logits to probabilities
        probabilities = 1 / (1 + np.exp(-predictions))
        return np.column_stack([1 - probabilities, probabilities])

    def predict(self, X):
        # Convert probabilities to class labels
        probabilities = self.predict_proba(X)[:, 1]
        return (probabilities >= 0.5).astype(int)

In [317]:
from sklearn.preprocessing import LabelEncoder

# Кодирование целевой переменной
label_encoder = LabelEncoder()
y_train_classification_encoded = label_encoder.fit_transform(y_train_classification)
y_test_classification_encoded = label_encoder.fit_transform(y_test_classification)

In [318]:
# Применение собственной реализации для классификации
rf_classifier = RANDOMFORESTCLASSIFIER(n_estimators=10, max_depth=3, min_samples_split=2, random_state=42)
rf_classifier.fit(X_train_preprocessed, y_train_classification_encoded)

my_train_predict_classification = rf_classifier.predict(X_train_preprocessed)
my_test_predict_classification = rf_classifier.predict(X_test_preprocessed)

In [319]:
# Расчет метрик
my_train_f1_classification = f1_score(y_train_classification_encoded, my_train_predict_classification, average='micro')
my_train_accuracy_classification = accuracy_score(y_train_classification_encoded, my_train_predict_classification)
my_train_precision_classification = precision_score(y_train_classification_encoded, my_train_predict_classification, average='micro')
my_train_recall_classification = recall_score(y_train_classification_encoded, my_train_predict_classification, average='micro')

my_test_f1_classification = f1_score(y_test_classification_encoded, my_test_predict_classification, average='micro')
my_test_accuracy_classification = accuracy_score(y_test_classification_encoded, my_test_predict_classification)
my_test_precision_classification = precision_score(y_test_classification_encoded, my_test_predict_classification, average='micro')
my_test_recall_classification = recall_score(y_test_classification_encoded, my_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', my_train_f1_classification, my_train_accuracy_classification, my_train_precision_classification, my_train_recall_classification)
print('Test', my_test_f1_classification, my_test_accuracy_classification, my_test_precision_classification, my_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.6919385796545106 0.6919385796545106 0.6919385796545106 0.6919385796545106
Test 0.6781609195402298 0.6781609195402298 0.6781609195402298 0.6781609195402298


In [320]:

# Применение собственной реализации для регрессии
rf_regressor = RANDOMFORESTREGRESSOR(n_estimators=10, max_depth=3, min_samples_split=2, random_state=42)
rf_regressor.fit(X_train_preprocessed, y_train_regression)

my_train_predict_regression = rf_regressor.predict(X_train_preprocessed)
my_test_predict_regression = rf_regressor.predict(X_test_preprocessed)

In [321]:
# Расчет метрик
my_train_mse_regression = mean_squared_error(y_train_regression, my_train_predict_regression)
my_train_mae_regression = mean_absolute_error(y_train_regression, my_train_predict_regression)

my_test_mse_regression = mean_squared_error(y_test_regression, my_test_predict_regression)
my_test_mae_regression = mean_absolute_error(y_test_regression, my_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', my_train_mse_regression, my_train_mae_regression)
print('Test', my_test_mse_regression, my_test_mae_regression)

Метрика:          MSE            MAE
Train 163538.69151591312 284.32550329089844
Test 283285.619473642 321.77501410352886


Посмотрим работу алгоритма на улучшенном датасете

In [322]:
# Применение собственной реализации для классификации
dt_classifier = RANDOMFORESTCLASSIFIER(n_estimators=10, max_depth=3, min_samples_split=2, random_state=42)
dt_classifier.fit(X_train, y_train_classification_encoded)

new_my_train_predict_classification = dt_classifier.predict(X_train)
new_my_test_predict_classification = dt_classifier.predict(X_test)

In [323]:
# Расчет метрик
new_my_train_f1_classification = f1_score(y_train_classification_encoded, new_my_train_predict_classification, average='micro')
new_my_train_accuracy_classification = accuracy_score(y_train_classification_encoded, new_my_train_predict_classification)
new_my_train_precision_classification = precision_score(y_train_classification_encoded, new_my_train_predict_classification, average='micro')
new_my_train_recall_classification = recall_score(y_train_classification_encoded, new_my_train_predict_classification, average='micro')

new_my_test_f1_classification = f1_score(y_test_classification_encoded, new_my_test_predict_classification, average='micro')
new_my_test_accuracy_classification = accuracy_score(y_test_classification_encoded, new_my_test_predict_classification)
new_my_test_precision_classification = precision_score(y_test_classification_encoded, new_my_test_predict_classification, average='micro')
new_my_test_recall_classification = recall_score(y_test_classification_encoded, new_my_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', new_my_train_f1_classification, new_my_train_accuracy_classification, new_my_train_precision_classification, new_my_train_recall_classification)
print('Test', new_my_test_f1_classification, new_my_test_accuracy_classification, new_my_test_precision_classification, new_my_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.7351247600767754 0.7351247600767754 0.7351247600767754 0.7351247600767754
Test 0.7279693486590039 0.7279693486590039 0.7279693486590039 0.7279693486590039


In [324]:

# Применение собственной реализации для регрессии
dt_regressor = RANDOMFORESTREGRESSOR(n_estimators=10, max_depth=3, min_samples_split=2, random_state=42)
dt_regressor.fit(X_train, y_train_regression)

new_my_train_predict_regression = dt_regressor.predict(X_train)
new_my_test_predict_regression = dt_regressor.predict(X_test)

In [325]:
# Расчет метрик
new_my_train_mse_regression = mean_squared_error(y_train_regression, new_my_train_predict_regression)
new_my_train_mae_regression = mean_absolute_error(y_train_regression, new_my_train_predict_regression)

new_my_test_mse_regression = mean_squared_error(y_test_regression, new_my_test_predict_regression)
new_my_test_mae_regression = mean_absolute_error(y_test_regression, new_my_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', new_my_train_mse_regression, new_my_train_mae_regression)
print('Test', new_my_test_mse_regression, new_my_test_mae_regression)

Метрика:          MSE            MAE
Train 152340.68069261283 276.12074731189637
Test 199726.69677254785 282.3571925846317


##### Сравнение стандартной и собственной реализации Случайного леса до улучшений:
Стандартная реализация показывает высокие значения F1 и Accuracy на обучающей выборке, указывающие на эффективное обучение, однако заметно значительное снижение метрик на тестовой выборке (F1: 0.858), что указывает на переобучение.
Собственная реализация имеет более простую структуру, метрики обучения и тестирования близки, что указывает на отсутствие переобучения, но метрики (F1: 0.692) ниже, чем у стандартной реализации. Модель имеет очень высокие значения MSE/MAE, особенно на тестовой выборке, что демонстрирует слабую способность модели к точному предсказанию.

##### Сравнение стандартной и собственной реализации Случайного леса после улучшения бейзлайна:
Стандартная реализация после улучшений показывает значительное улучшение обучения (F1: 0.997), модель стала более сложной. Снижение F1 на тестовой выборке (0.808) может указывать на переобучение из-за чрезмерной сложности.
Собственная реализация демонстрирует улучшение всех метрик (F1: 0.735), модель стала более сбалансированной, но она всё ещё отстаёт от стандартной реализации, что связано с более простой логикой и отсутствием оптимизаций. MSE/MAE уменьшились, что указывает на повышение точности предсказаний.




### Вывод:
Библиотечная реализация обеспечивает высокую точность и производительность благодаря продвинутым алгоритмам и оптимизациям. Однако для достижения лучшего баланса между обучением и тестированием требует тщательной настройки гиперпараметров.

Собственная реализация после улучшений показывает достойные результаты, в некоторых аспектах приближаясь к библиотечной версии. Однако разрыв в точности и эффективности всё ещё остаётся заметным, что объясняется более простой архитектурой и отсутствием некоторых функциональных возможностей.